This Program will contain all the functions defined from previous workbooks. 

In [1]:
#First before any functions can be defined, We must import any libraries that may be needed. 

import glob
import pandas as pd
import numpy as np
import os
import cmath
import math
import sympy
from sympy import Symbol
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as tfl
from keras.models import load_model
import random
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import pickle
import tensorflow as tf
from tensorflow.keras import layers as tfl
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import time
import statistics
from matplotlib.ticker import FuncFormatter
import statsmodels.api as sm
from scipy import stats
from scipy.integrate import simps



First we will define functions that can randomly shuffle lists and/or arrays. This will help randomize future data

In [16]:
# Now we will define a function that can shuffle the contents of a list. 

def shuffle_lists(*lists):
    # Combine the lists using zip
    combined = list(zip(*lists))
    
    # Shuffle the combined list
    random.shuffle(combined)
    
    # Unzip the combined list back into separate lists
    shuffled_lists = list(zip(*combined))
    
    # Convert tuples back to lists
    return [list(shuffled_list) for shuffled_list in shuffled_lists]

#Now we will define a similar function to shuffle numpy arrays:

def unison_shuffled_copies(*arrays):
    # Ensure all input arrays have the same length
    lengths = [len(arr) for arr in arrays]
    assert all(length == lengths[0] for length in lengths), "All arrays must have the same length"
    
    # Generate a random permutation of indices
    p = np.random.permutation(lengths[0])
    
    # Shuffle each array according to the permutation
    return tuple(arr[p] for arr in arrays)




This next section will cover the functions defined to generate the Cody-Lorentz oscillator. All of these functions are describing terms from:

Ferlauto, A. S., et al. "Analytical model for the optical functions of amorphous semiconductors from the near-infrared to ultraviolet: Applications in thin film photovoltaics." Journal of Applied Physics 92.5 (2002): 2424-2436.

In [19]:
######## Function from the Cody-Loretnz Workbook ##############

def L(E, Amp, Br, Eo ): # From equation 1b
    
    L = (Amp * Br * Eo * E) / ( (E**2 - Eo**2)**2 + ( Br**2 * E**2 ) ) 
    
    return (L)  

def Z(Eo, Br): # From equation 9
    
    Z = ( Eo**2 - ((Br**2)/2)  )**(1/2) 
    return(Z)

def X(Eo, Br):  # From equation 10
    
    X = ( 4*Eo**2 - Br**2 )**(1/2)
    return(X)

def Gt(E, Eg): # From equation 4
    
    Gt = (E - Eg)**2 / (E**2)
    return(Gt)

def Ld(E, Eo, Br):  # From equation 17
    
    Ld = (E**2 - Eo**2)**2 + ((Br**2)*(E**2))
    return(Ld)

def Cot(E, Eg, Eo, Br): # From equation 11
    
    gt = Gt(E, Eg)
    ld = Ld(E, Eo, Br)
    Cot = (E * gt)/(2 * ld)
    
    return(Cot)

def Dot(E, Eg, Eo, Br): # From equation 12
    
    ld = Ld(E, Eo, Br)
    Dot = -(E + Eg)**2 / (2*E*ld)
    return(Dot)

def A3t(E, Eg, Eo, Br):# From equation 13
    
    cot = Cot(E, Eg, Eo, Br)
    dot = Dot(E, Eg, Eo, Br)
    a3t = -(cot + dot)
    return(a3t)

def A2t(E, Eg, Eo, Br): # From equation 14
    
    cot = Cot(E, Eg, Eo, Br)
    dot = Dot(E, Eg, Eo, Br)
    a2t = -E*(cot - dot)
    return(a2t)

def A1t(E, Eg, Eo, Br): # From equation 15
    
    cot = Cot(E, Eg, Eo, Br)
    dot = Dot(E, Eg, Eo, Br)
    z = Z(Eo, Br)
    a1t = -(E**2 - 2*(z**2))*(cot + dot)
    return(a1t)

def Aot(E, Eg, Eo, Br): # From equation 16
    
    cot = Cot(E, Eg, Eo, Br)
    dot = Dot(E, Eg, Eo, Br)
    z = Z(Eo, Br)
    aot = 1 - (  E*( E**2 - 2*(z**2) )*(cot - dot) )
    return(aot)


def I1t(Eo, Br, Et): # From equation 6

    z = Z(Eo, Br)
    x = X(Eo, Br)
    
    I1t = (1/(2*x*Br)) * (math.pi - 2*math.atan( 2*((Et**2 - z**2)/(x*Br)) )  ) 
    
    return(I1t)


def Ioat(Eo, Br, Et): # From equation 7
    
    x = X(Eo, Br)   
    Ioat = (1/(2*Br)) * ( math.pi - math.atan(((2*Et + x)/Br))  + math.atan(((-2*Et + x)/Br))  )
    return(Ioat)
    
def Iobt(Eo, Br, Et): # From equation 8
    
    x = X(Eo, Br)
    Iobt = (1/(4*x)) * math.log( (Et**2 + Eo**2 + x*Et) / (Et**2 + Eo**2 - x*Et)   )
    return(Iobt)
    
    
def ITL(E, Eg, Eo, Br, A, Et):# From equations 5-8

    
    aot = Aot(E, Eg, Eo, Br)
    a1t = A1t(E, Eg, Eo, Br)
    a2t = A2t(E, Eg, Eo, Br)
    a3t = A3t(E, Eg, Eo, Br)
    cot = Cot(E, Eg, Eo, Br)
    dot = Dot(E, Eg, Eo, Br)
    z = Z(Eo, Br)
    ld = Ld(Et, Eo, Br)
    i1t = I1t(Eo, Br, Et)
    ioat = Ioat(Eo, Br, Et)
    iobt = Iobt(Eo, Br, Et)
    
    ITL_Coeff = (2*A*Eo*Br)/math.pi
    ITL_t1 = a3t * ((z**2)*i1t - math.log((ld)**(1/4)))
    ITL_t2 = a2t * (ioat + iobt)
    ITL_t3 = a1t*i1t
    ITL_t4 = aot*( (ioat - iobt) / (Eo**2))
     
    try:
        ITL_t5 = cot * math.log(abs(E - Et))
    
    except ValueError:
        
        ITL_t5 = 0
        
    
    
    ITL_t6 = dot*math.log(E + Et)
    
    ITL = ITL_Coeff * (ITL_t1 + ITL_t2 + ITL_t3 + ITL_t4 - ITL_t5 - ITL_t6)
    
    return(ITL)
    
    



def Fsq(Ep, Eg): # From equations 29
    
    Fsq = Ep**2 + Eg**2
    return(Fsq)


def Ksq(Ep, Eg, Eo, Br): # From equations 30
    
    z = Z(Eo, Br)
    fsq = Fsq(Ep, Eg)
    
    Ksq = 2*fsq + 2*(z**2) - 4*Eg**2
    return(Ksq)

def Ycube(Ep, Eg, Eo, Br):# From equations 31
    
    fsq = Fsq(Ep, Eg)
    ksq = Ksq(Ep, Eg, Eo, Br)
    Ycube = Eo**4 + fsq*(ksq - fsq) - 4*(Eg**2)*ksq
    return(Ycube)

def Gc(E, Eg, Ep):# From equations 18
    
    Gc = (E-Eg)**2 / ( (E-Eg)**2 + Ep**2   )
    return(Gc)


def Ioc(Ep, Eg, Et): # From equations 20
    
    Ioc = (1 / Ep) * ( (math.pi/2) - math.atan( (Et - Eg)/Ep) )
    return(Ioc)

def Coc(E, Eg, Ep, Eo, Br): # From equations 21
    
    gc = Gc(E, Eg, Ep)
    ld = Ld(E, Eo, Br)
    coc = (E*gc) / (2*ld)
    return(coc)

def Doc(E, Eg, Ep, Eo, Br): # From equations 22
    
    ld = Ld(E, Eo, Br)
    doc =  -  (E*(E+Eg)**2) / ( (2*ld)*((E+Eg)**2 + Ep**2) )
    return(doc)

def Boc(E, Ep, Eg, Eo, Br): # From equations 23
    
    ycube = Ycube(Ep, Eg, Eo, Br)
    fsq = Fsq(Ep, Eg)
    ksq = Ksq(Ep, Eg, Eo, Br)
    ld = Ld(E, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    
    boc_numerator = (ycube*fsq) * (   (ld*( (1/E)*(coc - doc) + 2*Eg*ksq*(1/ycube)*(coc + doc) )) - (1)  )
    
    boc_denominator = (ksq - fsq)*fsq*ycube + (Eo**4)*ycube + 4*(Eg**2)*fsq*(ksq**2)
    
    boc = boc_numerator / boc_denominator
    
    return(boc)

def B1c(E, Ep, Eg, Eo, Br): # From equations 24
    
    boc = Boc(E, Ep, Eg, Eo, Br)
    ksq = Ksq(Ep, Eg, Eo, Br)
    ycube = Ycube(Ep, Eg, Eo, Br)
    ld = Ld(E, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    
    b1c = (1/ycube) * ( (2*Eg*ksq*boc) - (ld*(coc + doc))  )
    return(b1c)

def A3c(E, Ep, Eg, Eo, Br): # From equations 25
    
    b1c = B1c(E, Ep, Eg, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    
    a3c = -(b1c + coc + doc)
    return(a3c)

def A2c(E, Ep, Eg, Eo, Br): # From equations 26
    
    boc = Boc(E, Ep, Eg, Eo, Br)
    b1c = B1c(E, Ep, Eg, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    
    a2c = - ( boc + 2*Eg*b1c + E*(coc - doc) )
    return(a2c)

def A1c(E, Ep, Eg, Eo, Br): # From equations 27
    
    boc = Boc(E, Ep, Eg, Eo, Br)
    ksq = Ksq(Ep, Eg, Eo, Br)
    fsq = Fsq(Ep, Eg)
    b1c = B1c(E, Ep, Eg, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    z = Z(Eo, Br)
    
    a1c = - ( 2*Eg*boc - (ksq - fsq)*b1c + (E**2 - 2*(z**2))*(coc + doc) )
    return(a1c)

def Aoc(E, Ep, Eg, Eo, Br): # From equations 28
    
    ksq = Ksq(Ep, Eg, Eo, Br)
    fsq = Fsq(Ep, Eg) 
    boc = Boc(E, Ep, Eg, Eo, Br)
    b1c = B1c(E, Ep, Eg, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    z = Z(Eo, Br)
    
    aoc = 1 + (ksq - fsq)*boc + 2*Eg*ksq*b1c - E*((E**2) - 2*(z**2))*(coc - doc)
    return(aoc)



def ITL_C(E, Ep, Eg, Eo, Br, A, Et): # From equations 19
    
    aot = Aoc(E, Ep, Eg, Eo, Br)
    a1t = A1c(E, Ep, Eg, Eo, Br)
    a2t = A2c(E, Ep, Eg, Eo, Br)
    a3t = A3c(E, Ep, Eg, Eo, Br)
    cot = Coc(E, Eg, Ep, Eo, Br)
    dot = Doc(E, Eg, Ep, Eo, Br)
    z = Z(Eo, Br)
    ld = Ld(Et, Eo, Br)
    i1t = I1t(Eo, Br, Et)
    ioat = Ioat(Eo, Br, Et)
    iobt = Iobt(Eo, Br, Et)
    
    ITL_Coeff = (2*A*Eo*Br)/math.pi
    ITL_t1 = (a3t*((z**2)*i1t - math.log((ld)**(1/4))))
    ITL_t2 = (a2t*(ioat + iobt))
    ITL_t3 = a1t*i1t
    ITL_t4 = aot*( (ioat - iobt) / (Eo**2))
    
    try:
        ITL_t5 = cot * math.log(abs(E - Et))
    
    except ValueError:
        
        ITL_t5 = 0
    
    
    ITL_t6 = dot*math.log(E + Et)
    
    ITL = ITL_Coeff * (ITL_t1 + ITL_t2 + ITL_t3 + ITL_t4 - ITL_t5 - ITL_t6)
    
    return(ITL)


def ICL(E, Ep, Eg, Eo, Br, A, Et): # From equations 19-22
    
    iTL = ITL_C(E, Ep, Eg, Eo, Br, A, Et)
    ioc = Ioc(Ep, Eg, Et)
    boc = Boc(E, Ep, Eg, Eo, Br)
    b1c = B1c(E, Ep, Eg, Eo, Br)
    
    ICL = iTL + (2*A*Eo*Br)/math.pi * (  b1c*( Eg*ioc - math.log( ((Et-Eg)**2 + Ep**2)**(1/2) ) ) + (boc*ioc) )
    
    return(ICL)




def Get_TL_Material(E, Eg, Eo, Br, Amp, Egt, E_inf, wv):

    # generates a Tauc-Lorentz dielectic function based on the provided spectra (E, wv) and parameters (Eg, Eo, Br, Amp, Egt, E_inf)

    
    e2 = [] # values for the imaginary part of dielectic function 
    e1 = [] # values for the real part of dielectic function
    e = [] # values for the complete dielectric function
    n = [] # refractive index 
    k = [] # extinction coefficient
    N = [] # complex refractive index



    # Iterate over the entire spectral range 
    for i in range(len(E)):

        #Calculate e2
    #***************************************************
        #calculate e2 if E <= Egt
        if E[i] <= Egt:
            #calculate e2 
            e2_temp = 0 # term is 0 in this range when neglective the Urbach Energy

            #store e2
            e2.append(e2_temp) 

        # calculate e2 if E > Egt   
        else:      
            #calculate e2   
            g = Gt(E[i], Eg) # calculates G        
            l = L(E[i], Amp, Br, Eo) # calculates L        
            e2_temp = g * l # e2 = G * L when E > Egt        
            # store e2
            e2.append(e2_temp) 



         #Calculate e1
    #***************************************************
        ITL_temp = ITL(E[i], Eg, Eo, Br, Amp, Egt) #TL analytic solution to KK-integration

        e1_temp = E_inf + ITL_temp # add E_infinity

        e1.append(e1_temp) # store result

         #Calculate e
    #***************************************************
        e.append( complex(e1_temp, e2_temp) )

         #Calculate n
    #***************************************************
        n.append( cmath.sqrt(0.5 * (e1_temp + cmath.sqrt(e1_temp**2 + e2_temp**2 ))).real )

         #Calculate k
    #***************************************************    
        k.append(cmath.sqrt(0.5 * (- e1_temp + cmath.sqrt(e1_temp**2 + e2_temp**2  ))).real)

             #Calculate N
    #***************************************************    
        N.append( complex(n[i], k[i]) )

    # save values
    dict = {'Wavelength (nm)':wv , 'Energy (eV)': E, 'e1': e1, 'e2':e2, 'e':e, 'n':n, 'k':k, 'N': N}

    df = pd.DataFrame(dict)
    df.name = "Eg_" + str(Eg) + "_Eo_" + str(Eo) + "_Br_" + str(Br) + "_Amp_" + str(Amp) + "_Et_" + str(Egt) + "_Einf_" + str(E_inf) 

    #return values
    return(df)




# Now its time to turn the simulated Cody-Lorentz oscillator into a material dataframe:

def Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv):

    # generates a Cody-Lorentz dielectic function based on the provided spectra (E, wv) and parameters (Ep, Eg, Eo, Br, Amp, Egt, E_inf)
    
    e2 = [] # values for the imaginary part of dielectic function 
    e1 = [] # values for the real part of dielectic function
    e = [] # values for the complete dielectric function
    n = [] # refractive index 
    k = [] # extinction coefficient
    N = [] # complex refractive index


    # Iterate over the entire spectral range 
    for i in range(len(E)):

        #Calculate e2
    #***************************************************
        #calculate e2 if E <= Egt
        if E[i] <= Egt:
            #calculate e2 
            e2_temp = 0 # term is 0 in this range when neglective the Urbach Energy

            #store e2
            e2.append(e2_temp) 

        # calculate e2 if E > Egt   
        else:      
            #calculate e2   
            g = Gc(E[i], Eg, Ep) # calculates G        
            l = L(E[i], Amp, Br, Eo) # calculates L        
            e2_temp = g * l # e2 = G * L when E > Egt        
            # store e2
            e2.append(e2_temp) 

         #Calculate e1
    #***************************************************
        ICL_temp = ICL(E[i], Ep, Eg, Eo, Br, Amp, Egt) #Cody-Loretnz analytic solution to KK-integration

        e1_temp = E_inf + ICL_temp # add E_infinity

        e1.append(e1_temp) # store result

         #Calculate e
    #***************************************************
        e.append( complex(e1_temp, e2_temp) )

         #Calculate n
    #***************************************************
        n.append( cmath.sqrt(0.5 * (e1_temp + cmath.sqrt(e1_temp**2 + e2_temp**2 ))).real )

         #Calculate k
    #***************************************************    
        k.append(cmath.sqrt(0.5 * (- e1_temp + cmath.sqrt(e1_temp**2 + e2_temp**2  ))).real)

             #Calculate N
    #***************************************************    
        N.append( complex(n[i], k[i]) )

    #Now its time to store results into a pandas data frame
    
    
    # Store data 
    dict = {'Wavelength (nm)':wv , 'Energy (eV)': E, 'e1': e1, 'e2':e2, 'e':e, 'n':n, 'k':k, 'N': N}

    df = pd.DataFrame(dict)
    df.name = "Ep_" + str(Ep) + "_Eg_" + str(Eg) + "_Eo_" + str(Eo) + "_Br_" + str(Br) + "_Amp_" + str(Amp) + "_Et_" + str(Egt) + "_Einf_" + str(E_inf) 

    # return data
    return(df)














The next function will be the function that defines the Bruggeman Effective Medium Approximation. 

In [22]:
#Now generate EMA function

def Bruggeman_EMA(Material1, Material2, Material2_Fraction):
    
    """
    Material1 and Material2 are both pandas dataframs with columns labeled ['e'] which corrispond to the complex dielectic material of the materials
    
    Material2_fraction is a number in the range of [0:1]
    """

    
    B = ( -1 * Material1['e'] * Material2_Fraction + 2 * Material2['e'] * Material2_Fraction - Material2['e'] * (1 - Material2_Fraction ) + 2 * Material1['e'] * (1 - Material2_Fraction ))
    
    eff =[] # dielectric of the effective material
    e1 =[] #  real part of the dielectic function
    e2 =[] #  imaginary part of the dielectic function
    n = [] # refractive index 
    k = [] # extinction coefficient
    N = [] # complex refractive index
    
    for i in range(len(B)): # iterate over spectral range


        t = (-B[i] + cmath.sqrt(B[i] * B[i] - 4 * (-2) *  Material1['e'][i] *  Material2['e'][i]) ) / -4

        if (t.imag > 0 ):
            eff.append(t)
            e1.append(t.real)
            e2.append(t.imag)
            nt = cmath.sqrt(0.5 * (t.real + cmath.sqrt(t.real*t.real + t.imag * t.imag ))).real
            kt = cmath.sqrt(0.5 * (- t.real + cmath.sqrt(t.real*t.real + t.imag * t.imag ))).real
            n.append(nt)
            k.append(kt)
            N.append(complex(nt, kt))

        else:

            t = (-B[i] - cmath.sqrt(B[i] * B[i] - 4 * (-2) *  Material1['e'][i] *  Material2['e'][i]) ) / -4

            eff.append(t)
            e1.append(t.real)
            e2.append(t.imag)
            nt = cmath.sqrt(0.5 * (t.real + cmath.sqrt(t.real*t.real + t.imag * t.imag ))).real
            kt = cmath.sqrt(0.5 * (- t.real + cmath.sqrt(t.real*t.real + t.imag * t.imag ))).real
            n.append(nt)
            k.append(kt)
            N.append(complex(nt, kt))




    dict =  {'Wavelength (nm)': wv, 'e1': e1, 'e2': e2, 'n': n, 'k' : k, 'N' : N} 
    df = pd.DataFrame(dict)
    
    df.name =  Material1.name + "_" + str(round(1 - Material2_Fraction, 2)) + "_"+ Material2.name + "_" + str(Material2_Fraction)
    
    
    return(df)
    


In [ ]:
def Bruggeman_EMA_Roussel(Material1, Material2, c):

    """
    This version of the Bruggeman EMA from Roussel et all
    
    Material_1 and Material_2 should be pandas data frames with columns labeled ['e'] which corrispond to the complex dielectic function of the materials.
    c should be the material fraction of Material2. It should be a number in the range of [0,1]
    
    """

    wv = Material1['Wavelength (nm)'].to_numpy()


    # Step 1: Define p (eq 8 in paper)

    N1 = Material1['N'].to_numpy()
    N2 = Material2['N'].to_numpy()

    p = N1 / N2

    # Step 2: Define b (eq 12 in paper)

    b = 0.25 * ( (3 * c - 1) * ( ( 1 / p ) - p ) + p )

    # Step 3: Define z (eq 13 in paper) 

    z = b + (b**2 + 0.5)**(1/2)

    # Step 4: convert z into dielectic function

    e = z * N1 * N2 # eq 6

    e1 =  e.real
    e2 =  e.imag
    
    n = np.sqrt(0.5 * (e1 + np.sqrt(e1**2 + e2**2))).real

    k = np.sqrt(0.5 * (-e1 + np.sqrt(e1**2 + e2**2))).imag

    N = n + 1j* k


    df = pd.DataFrame({
    'Wavelength (nm)': wv,
    'e': e,
    'e1': e1,
    'e2': e2,
    'n': n,
    'k': k,
    'N': N
     })

    df.name =  Material1.name + "_" + str(round(1 - c, 2)) + "_"+ Material2.name + "_" + str(c)
    
    return (df)
    
    




The next function will define the unweighted error function used to compare the model generated and measured ellipsometric spectra

In [25]:

def SE_Sigma(array, df, m):
    """
    Calculate the unweighted error function between the measured ellipsometric spectra and the simulated spectra
    
    Parameters:
    - array: NumPy array - Measured ellipsometric spectra
    - df: Pandas DataFrame - Simulated ellipsometric spectra
    - m: number of fit parameters for optical and structural model
    
    Returns:
    - Sigma: The unweighted error function
    """
    # Ensure the array has the same shape as the DataFrame
    if array.shape != df.shape:
        raise ValueError("NumPy array and DataFrame must have the same shape")
    
    # Convert the NumPy array to a DataFrame for easier handling
    n = array.shape[0] * array.shape[1]
    array_df = pd.DataFrame(array, columns=df.columns, index=df.index) 
    array_df.columns = ['N', 'C', 'S']
    
    # Calculate Sigma

    #Calculate (N - N')^2  we will call this variable N_diff
    N_diff = ( (df['N'] - array_df['N'])**2 ).sum()

    #Calculate (C - C')^2  we will call this variable C_diff
    C_diff = ( (df['C'] - array_df['C'])**2 ).sum()

    #Calculate (S - S')^2  we will call this variable S_diff
    S_diff = ( (df['S'] - array_df['S'])**2 ).sum()

    #Now we calculate n
    n = array.shape[0] # should be 697 for the intended spectral range

    #Now Put everything together to get Sigma

    Sigma = ( (1/(3*n - m)) * (N_diff + C_diff + S_diff ) )**(1/2)

    return Sigma    



The next function(s) will generate ellipsometric specta.

In [ ]:
def Snells_Law(Structure, AOI_degrees):
    """
    Compute the angle of light in each material using Snell's Law.

    Parameters:
    - Structure: list of pandas DataFrames, each with a column 'N' representing refractive index.
    - AOI_degrees: scalar angle of incidence in degrees (applies equally to all data points).

    Returns:
    - angles_deg: 2D NumPy array of angles (in degrees), shape (num_layers, num_points)
    """

    # Step 1: Extract refractive indices into a 2D array: shape (num_layers, num_points)
    N = np.stack([
    df["N"].to_numpy()
    if "N" in df.columns
    else df["n"].to_numpy() + 1j * df["k"].to_numpy()
    if all(col in df.columns for col in ["n", "k"])
    else np.full(df.shape[0], np.nan + 1j * np.nan)  # or raise an error
    for df in Structure
    ])

    num_layers, num_points = N.shape

    # Step 2: Initialize angle array: same shape as N
    angles = np.zeros((num_layers, num_points), dtype = complex)

    # Step 3: Convert AOI to radians and broadcast it across the first layer
    AOI_rad = np.radians(AOI_degrees)
    angles[0] = AOI_rad  # fills first row with scalar AOI value

    # Step 4: Apply Snell's Law across each interface
    for i in range(1, num_layers):
        ratio = N[i-1] / N[i]
        sin_theta = ratio * np.sin(angles[i-1])

        angles[i] = np.arcsin(sin_theta)

    # Step 5: Convert to degrees (if desired)
    # angles_deg = np.degrees(angles)

    return angles

In [ ]:
def fresnel_coefficients(N, angles):
    """
    Compute Fresnel reflection and transmission coefficients for s and p polarizations at each interface.

    Parameters:
    - N: (num_layers, num_points) array of refractive indices
    - angles: (num_layers, num_points) array of angles in radians

    Returns:
    - rs, rp: reflection coefficients (s and p polarization)
    - ts, tp: transmission coefficients (s and p polarization)
    """

    # Slice interface values
    n1, n2 = N[:-1], N[1:]
    theta1, theta2 = angles[:-1], angles[1:]

    # Cosines
    cos1 = np.cos(theta1)
    cos2 = np.cos(theta2)

    # Avoid divide-by-zero if needed
    denominator_s = (n1 * cos1 + n2 * cos2)
    denominator_p = (n2 * cos1 + n1 * cos2)

    # s-polarized
    rs = (n1 * cos1 - n2 * cos2) / denominator_s
    ts = (2 * n1 * cos1) / denominator_s

    # p-polarized
    rp = (n2 * cos1 - n1 * cos2) / denominator_p
    tp = (2 * n1 * cos1) / denominator_p

    return rs, rp, ts, tp

In [ ]:
def Scattering_Matrix(N, angles, d, wavelengths, r, t):

    L, P = N.shape # L = number of layers, P = spectral points

    d = d[:, np.newaxis]
    λ = wavelengths[np.newaxis, :]
    # Calculate exponential term "E"
    E = (2 * np.pi / λ) * N[1:-1] * d * np.cos(angles[1:-1])
    #First define the propagation matrices. The light will propagate through L - 2 films, discounting the ambient and the substrate.
    prop_matrices = np.zeros((L - 2, P, 2, 2), dtype=complex)
    # Fill the diagonals
    prop_matrices[:, :, 0, 0] = np.exp(-1j * E)    
    prop_matrices[:, :, 1, 1] = np.exp(1j * E)    

    #Now define the interface matrices. There will be L -1 interfaces. 
    int_matrices = np.zeros((L - 1, P, 2, 2), dtype=complex)
    int_matrices[:, :, 0, 0] = 1 / t
    int_matrices[:, :, 0, 1] =  r / t
    int_matrices[:, :, 1, 0] =  r / t
    int_matrices[:, :, 1, 1] =  1 / t

    # Initialize the result array
    S = int_matrices[0]  # Start with Y0
    
    # Perform the matrix multiplication in the specified order
    for i in range(1, L - 1):
        #print('test')
        S = np.matmul(S, prop_matrices[i - 1])  # Multiply with X layers
        S = np.matmul(S, int_matrices[i])  # Multiply with Y layers

    return(S)

In [ ]:
def SE_Sim(Structure, AOI, d, write_data=False, NCS=True):


    # Step 1: Extract the complex index of refraction (N) from each column in structure. 
    wv = Structure[0]['Wavelength (nm)'].to_numpy()
    
    N = np.stack([
    df["N"].to_numpy()
    if "N" in df.columns
    else df["n"].to_numpy() + 1j * df["k"].to_numpy()
    if all(col in df.columns for col in ["n", "k"])
    else np.full(df.shape[0], np.nan + 1j * np.nan)  # or raise an error
    for df in Structure
    ])

    # Step 2: Calculate angles from Snells law.
    angles =  Snells_Law(Structure, AOI)

    #print(angles.shape)

    # Step 3: Calculate the fresnel coefficients at each interface
    rs, rp, ts, tp = fresnel_coefficients(N, angles)
    #print(ts)

    # Step 4: Calculate S matrix 
    Ss = Scattering_Matrix(N, angles, d, wv, rs, ts)
    Sp = Scattering_Matrix(N, angles, d, wv, rp, tp)

   # Step 5: Calculate Rp and Rs
    Rp = (Sp[:, 1, 0] / Sp[:, 0, 0])
    Rs = (Ss[:, 1, 0] / Ss[:, 0, 0]) 

    #Step 5.5 Calculate Rho (for some reason, complex conjugate is needed, like due to a convention mismatch in previous math)

    rho =  np.conj(Rp / Rs)

   # Step 6: Calculate psi and delta

    psi = np.arctan( np.abs( rho ) ).real

    delta =  np.unwrap(np.angle(rho)) 

    # Step 7: Calculate N, C, and S

    N = np.cos(2 * psi).real
    C = (np.sin(2 * psi) * np.cos(delta)).real
    S = (np.sin(2 * psi) * np.sin(delta)).real

    # Step 7.5 convet psi and delta to degress



    if NCS: 
    
        my_dict =  {'Wavelength (nm)': wv, 'N': N, 'C': C, 'S': S}
        df = pd.DataFrame(my_dict)

    if not NCS: 

        # Step 7.5 convet psi and delta to degress

        psi = psi * 180 / np.pi
        delta = delta * 180 / np.pi

        my_dict =  {'Wavelength (nm)': wv, 'Psi': psi, 'Delta': delta}
        df = pd.DataFrame(my_dict)


    if not write_data:
        
        return(df)

    if write_data: 
        name = str()
        for z in range(len(d)):
            name = name + str(Structure[z+1].name) + "_"
            name = name + str(d[z]) + "nm"
            if z < len(d) - 1:
                name = name + "_"

        name_row = pd.DataFrame([[name] + [''] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([name_row, df], ignore_index=True)
        df.name = name

        timestamp = datetime.now()
        timestamp_string = timestamp.strftime('%Y-%m-%d-%H-%M-%S') + f"-{timestamp.microsecond // 1000:03d}"
        title = "trial_" + timestamp_string  + ".csv"


        df.to_csv( title + ".csv" , index=False)
        return(df)

Now we will define a function that can load in stored data. One function will work with Cody-Loretnz generated data and the other with Tauc-Loretnz

In [1]:

def get_data_CL(path):

    os.chdir(r"C:\Users\bordo\Documents\UToledo\Research\ML\a-Si\OpProp")
    file =  r"C:\Users\bordo\Documents\UToledo\Research\ML\a-Si\OpProp\SLG.csv"
    SLG = pd.read_csv(file)
    SLG.name = 'SLG'
    E = SLG['Energy (eV)']

    os.chdir(path)
    files = glob.glob(path + "/*.csv") # Loads in the files for the 1st path


    Answer_Key_Bulk_Thickness = []
    Answer_Key_EMA_Thickness = []
    Answer_Key_NTVE_Thickness = []
    Answer_Key_AngOff = []

    Answer_Key_Voidfrac =[]

    Answer_Key_Einf =[]
    Answer_Key_Amp =[]
    Answer_Key_Et = []
    Answer_Key_Br =[]
    Answer_Key_Eo =[]
    Answer_Key_Eg =[]
    Answer_Key_Ep =[]
    Answer_Key_EMA_bool = []
    Answer_Key_EMA_thickness = []
    Answer_Key_Substrate =[]
    



    file_name = []
    data =[]

    #Answer_Key_AngOff =[]

    for i in range(len(files)): # Iterate for every simulated file


            #Load in data from CSV files
            df = pd.read_csv(files[i], index_col=None)

            #Get Answer Key from CSV file
            a = str(df.iloc[0, 0])

            
            #print(a)
            #Store Answer Key as "Filename"
            file_name.append(a)
            #print(a)
            # Remove Answer Key from the data frame to avoid NAN values
            df = df.iloc[1:].reset_index(drop=True)


            # Remove data that we do not want to train the Neural Network on. This may change with other iterations
            df = df.drop(df.columns[0], axis=1)
            df.insert(0, 'Energy (eV)', E)


            #stores the data the neural network will be trained on. (N,C,S in this case)
            data.append(df)


            #Splits the answer key (a) into different sections delimited by the "_" symbol
            a = a.split('\\')[-1].split('_')
            #print(len(a))
            #print(a)
            
            #Stores the Ep parameter for this data set
            Answer_Key_Ep.append(float(a[1]))
        
            #Stores the Eg parameter for this data set
            Answer_Key_Eg.append(float(a[3]))

            #Stores the Eo parameter for this data set
            Answer_Key_Eo.append(float(a[5]))

            #Stores the Br parameter for this data set
            Answer_Key_Br.append(float(a[7]))

            #Stores the Amp parameter for this data set
            Answer_Key_Amp.append(float(a[9]))

            #Stores the Einf parameter for this data set
            Answer_Key_Einf.append(float(a[13]))  


            if len(a) == 33: 
                
                Answer_Key_EMA_bool.append(0) # EMA 0 means no EMA
                Answer_Key_EMA_thickness.append(0)
                Answer_Key_Bulk_Thickness.append(float(a[14].split('nm')[0]))
                Answer_Key_Substrate.append(1) # 1 means soda-lime glass
    
            if len(a) == 51:

                Answer_Key_EMA_bool.append(1)  # EMA 1 means there is an EMA layer
                Answer_Key_EMA_thickness.append(float(a[17].split('nm')[0]))
                Answer_Key_Bulk_Thickness.append(float(a[32].split('nm')[0]))
                Answer_Key_Substrate.append(1) # 1 means soda-lime glass

            if len(a) == 18: 

                Answer_Key_EMA_bool.append(0) # EMA 0 means no EMA
                Answer_Key_EMA_thickness.append(0)
                Answer_Key_Bulk_Thickness.append(float(a[14].split('nm')[0]))
                Answer_Key_Substrate.append(0) # 0 means Si wafer
    
            if len(a) == 36:

                Answer_Key_EMA_bool.append(1)  # EMA 1 means there is an EMA layer
                Answer_Key_EMA_thickness.append(float(a[17].split('nm')[0]))
                Answer_Key_Bulk_Thickness.append(float(a[32].split('nm')[0]))
                Answer_Key_Substrate.append(0) # 0 means Si wafer



        



    train_data = np.array(data)
    train_files = np.array(file_name)
    train_label_Ep =np.array(Answer_Key_Ep)
    train_label_Eg =np.array(Answer_Key_Eg)
    train_label_Eo =np.array(Answer_Key_Eo)
    train_label_Br =np.array(Answer_Key_Br) 
    train_label_Amp =np.array(Answer_Key_Amp)
    train_label_Einf =np.array(Answer_Key_Einf)
    train_label_BulkT = np.array(Answer_Key_Bulk_Thickness)
    train_label_EMA_bool = np.array(Answer_Key_EMA_bool)
    train_label_EMA_Thickness = np.array(Answer_Key_EMA_thickness)
    train_label_substrate = np.array(Answer_Key_Substrate)
    

    return(
        train_files, 
        train_data,
        train_label_Ep,
        train_label_Eg,
        train_label_Eo, 
        train_label_Br, 
        train_label_Amp, 
        train_label_Einf, 
        train_label_BulkT,
        train_label_EMA_bool,
        train_label_EMA_Thickness,
        train_label_substrate
         )
